In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

In [2]:
# read in rules
df_rules = pd.read_csv("data/rules.csv", header = None, names = ['test_value','min','max'])
df_rules['test_value'].unique()

array([12,  5,  3], dtype=int64)

In [3]:
#put in ascending order for rules, reset_index or just set_index to make it lable index_able. 
# Save test_point
df_rules.sort_index(by=['test_value'], ascending=[True],inplace=True)
#df_rules.reset_index(inplace=True, drop=True)
test_point = df_rules['test_value'].tolist()
df_rules.set_index([test_point], inplace=True)
df_rules

C:\Users\Zhen Xu\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:3: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)
  app.launch_new_instance()


,test_value,min,max
3,3,2.8,3.2
5,5,4.7,5.3
12,12,11.4,12.4


In [5]:
# read in test data in csv format  names = test_point
df_test_data = pd.read_csv("data/result.csv", header = None)
df_test_data.head()

,0,1,2
0,12.364519,5.204825,3.101785
1,12.800000,4.975547,3.065432
2,11.874330,5.246546,2.874324
3,NaN,5.220957,3.103659
4,NaN,4.783890,2.979370


In [6]:
# order columns with acending average, or mean, nanmean ignoring NaNs, average with optional weight parameter not with NAN.

def order_test_data(df_test_data):
    # calculate average of each column, transfer to int
    average_test = df_test_data.apply(np.mean)
    average_test_int = list(map(lambda x: int(round(x)), average_test.tolist()))
    
    # use the average value as pandas column name
    df_test_data.columns = average_test_int
    print("~~~~~~~~~~~List of test points from test data~~~~~~~~~~~~~\n")
    print(sorted(average_test_int))
    
    #sort column with the sorted average
    df_test_data= df_test_data.ix[:, sorted(average_test_int)]
    return df_test_data

df_test_data = order_test_data(df_test_data)
df_test_data[0:10]

~~~~~~~~~~~List of test points from test data~~~~~~~~~~~~~

[3, 5, 12]


,3,5,12
0,3.101785,5.204825,12.364519
1,3.065432,4.975547,12.800000
2,2.874324,5.246546,11.874330
3,3.103659,5.220957,NaN
4,2.979370,4.783890,NaN
5,2.974575,5.026689,NaN
6,3.131787,4.732113,NaN
7,2.841922,4.948719,NaN
8,3.033848,5.141135,NaN
9,3.600000,4.819512,NaN


In [8]:
def result_output(df_test_data, df_rules, test_point):
    test_size = len(df_test_data)
    temp=0
    output_list=[]
    for j in test_point:
        
        # fill na with the test_point data, , inplace=True is not used
        df_test_data[j].fillna(j)
        for i in np.arange(test_size):
            if df_test_data[j][i] > df_rules.loc[j]['max'] or df_test_data[j][i] < df_rules.loc[j]['min']:
                temp = temp + 1
        output_list.append(temp)
        temp=0
    output_series = pd.Series(output_list, index = test_point)
    print('result outside the range')
    return output_series

In [9]:
result_output(df_test_data, df_rules, test_point)

result outside the range


3     2
5     2
12    1
dtype: int64

#check how many not match and it's location

In [10]:
Output_point = input("Enter the voltage you want to check: " + str(test_point));

Enter the voltage you want to check: [3, 5, 12]12


In [11]:
Output_point

'12'

In [12]:
def check_output_point(df_test_data, Output_point):
    Output_point = int(Output_point)
    error_result = df_test_data[(df_test_data[Output_point] > df_rules.loc[Output_point]['max']) | \
                                (df_test_data[Output_point] < df_rules.loc[Output_point]['min']) ][Output_point]
    error_count = len(error_result)
    
    print("Total error point:" + str(error_count) )
    return error_result

In [13]:
check_output_point(df_test_data, Output_point)

Total error point:1


1    12.8
Name: 12, dtype: float64

# Future add timestamp column in test to corelate the reason